# Bioinformatika Lab. 1

In [168]:
from Bio import SeqIO
from Bio.Seq import Seq
from math import ceil
from Bio.Alphabet import IUPAC
from Orf import Orf
from Bio.Alphabet import generic_dna, generic_rna

In [133]:
rec = SeqIO.read("plazmide.fasta", "fasta")
sequence = rec.seq

In [139]:
print(rec)

ID: plazmide
Name: plazmide
Description: plazmide
Number of features: 0
Seq('TTGCGCCGGCGAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATC...CCT', SingleLetterAlphabet())


In [140]:
start = ["ATG"]
stop = ["TAA", "TAG", "TGA"]
convert = {"C" : "G", "G" : "C", "T" : "A", "A" : "T"}

In [141]:
def findOrfs(data):
    orfstart = -1
    orfend = -1
    orfs = []
    for i in range(0,ceil(len(data)/3)):
        index = i * 3
        codon = data[index:index+3]
        if codon in start:
           # print("Start: {} at {}".format(codon, index))
            if orfstart > 0 and orfstart < orfend:
                value = data[orfstart:orfend]
                if len(value) >= 100:
                    orfs.append(Orf(orfstart, orfend, value))
            orfstart = index
        if codon in stop:
          #  print("Stop: {} at {}".format(codon, index))
            orfend = index+3
    return orfs

In [142]:
def complement(data):
    compl = []
    for var in data:
        compl.append(convert[var])
    return compl

In [143]:
def reverse(data):
    return list(reversed(data))

In [180]:
def getAllOrfs(data):
    orfs = findOrfs(data)
    orfs.extend(findOrfs(data[1:]))
    orfs.extend(findOrfs(data[2:]))
    orfs.extend(findOrfs(reverse(complement(data))))
    orfs.extend(findOrfs(reverse(complement(data))[1:]))
    orfs.extend(findOrfs(reverse(complement(data))[2:]))
    return orfs    

In [181]:
orfs = getAllOrfs(sequence) # mind that indexes are wrong for reversed sequences

In [182]:
def toRNAs(orfs):
    rnas = []
    for orf in orfs:
        rnas.append(toRNA(orf.value))
    return rnas

In [183]:
def toRNA(dna):
    conv = {"T" : "A", "A" : "U", "C" : "G", "G" : "C"}
    rna = ""
    for i in dna:
        rna += str(conv[i])
    return rna

In [196]:
def toProteins(rnas):
    proteins = []
    for rna in rnas:
        r = Seq(rna, generic_rna)
        proteins.append(r.translate())
    return proteins

In [198]:
rnas = toRNAs(orfs)

In [199]:
proteins = toProteins(rnas)